In [1]:
import os
import pandas as pd
import numpy as np
import json
from sklearn.utils import shuffle

In [16]:
read_dir='E:/文档/数据集/自适应实践数据集/matmat-2016-10/'
save_dir='../data/mat2016/'

answer_data=pd.read_csv(read_dir+'answers.csv',index_col=0)
item_data=pd.read_csv(read_dir+'items.csv')
skill_data=pd.read_csv(read_dir+'skills.csv')

prob_count_limit=15 #每个学生做过的习题下限

In [18]:
record=answer_data.loc[:,['student','item','correct']].drop_duplicates(subset=['student','item'],keep='first')\
    .dropna(axis=0,how='any')
record.columns=['user_id','item_id','score']

In [19]:
# 统计每个学生做了多少道题
problem_counter=record.groupby(by='user_id').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['item_id']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

record=record.set_index('user_id').loc[filtered_stu_id,:].reset_index()

有效的学生数： 6866


In [25]:
item_data=item_data.loc[:,['id','skill','skill_lvl_1','skill_lvl_2','skill_lvl_3']]\
    .drop_duplicates(keep='first')
item_data=item_data.melt(id_vars='id',value_vars=['skill','skill_lvl_1','skill_lvl_2','skill_lvl_3'])\
    .loc[:,['id','value']].drop_duplicates(keep='first').dropna(axis=0,how='any').astype('int')
item_data.columns=['item_id','knowledge_code']

In [27]:
skill=skill_data.loc[:,['id','parent']].fillna(-1)

In [28]:
item_unique=np.unique(record['item_id'])
know_unique=np.unique(item_data['knowledge_code'])
stu_unique=np.unique(record['user_id'])

In [29]:
stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))

In [30]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_data['item_id']=item_data['item_id'].map(item_old_new)
item_data['knowledge_code']=item_data['knowledge_code'].map(know_old_new)

skill['id']=skill['id'].map(know_old_new)
skill['parent']=skill['parent'].map(know_old_new)
skill=skill.dropna(axis=0,how='any').astype('int')
skill.columns=['knowledge_code','parent']

print('Done')

Done


In [32]:
item_open_df=item_data.set_index('item_id')

item_df=pd.DataFrame(columns=['item_id','knowledge_code'],index=range(1,len(item_unique)+1))
for item_i in range(1,len(item_unique)+1):
    item_df.loc[item_i,'item_id']=item_i
    item_df.loc[item_i,'knowledge_code']=np.array(item_open_df.loc[item_i,['knowledge_code']]).reshape(-1).tolist()

In [34]:
item_df.to_csv(save_dir+'item.csv',index=False)
record.to_csv(save_dir+'record.csv',index=False)
skill.to_csv(save_dir+'skill.csv',index=False)

In [36]:
print('学习者数：',len(stu_unique))
print('习题数：',len(item_unique))
print('知识点数：',len(know_unique))

学习者数： 6866
习题数： 1847
知识点数： 445
